# 03 - Recommender System

In [55]:
# imports
import pandas as pd
import numpy as np
import time

from scipy import sparse # cut down on memory size
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer

pd.options.display.max_columns = 35

___

In [2]:
# read in the data
df = pd.read_csv('../Data/cards_cleaned.csv')
df.head()

,name,layout,colors,color_identity,mana_cost,cmc,type_line,card_type,super_type,sub_type,oracle_text,oracle_text_token,legalities,rarity,power,toughness,loyalty,card_faces,activated_ability,triggered_ability,oracle_text_back,oracle_text_back_token,colors_back,power_back,toughness_back,loyalty_back,card_type_back,super_type_back,sub_type_back,mana_cost_back
0,Static Orb,normal,[],[],{3},3.0,Artifact,Artifact,NONE,NONE,"As long as Static Orb is untapped, players can...",as long as static orb is untapped players can'...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,Sensory Deprivation,normal,['U'],['U'],{U},1.0,Enchantment — Aura,Enchantment,NONE,Aura,Enchant creature Enchanted creature gets -3/-0.,enchant creature enchanted creature gets -3/-0,pioneer modern legacy pauper vintage penny com...,common,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,Road of Return,normal,['G'],['G'],{G}{G},2.0,Sorcery,Sorcery,NONE,NONE,Choose one — • Return target permanent card fr...,choose one return target permanent card from y...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,Storm Crow,normal,['U'],['U'],{1}{U},2.0,Creature — Bird,Creature,NONE,Bird,Flying (This creature can't be blocked except ...,flying this creature can't be blocked except b...,modern legacy pauper vintage penny commander duel,common,1,2,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,Walking Sponge,normal,['U'],['U'],{1}{U},2.0,Creature — Sponge,Creature,NONE,Sponge,{T}: Target creature loses your choice of flyi...,{t} target creature loses your choice of flyin...,legacy vintage commander duel,uncommon,1,1,NONE,NONE,1.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [3]:
df.shape

(18108, 30)

___

## Oracle text recommender system
To start I want to build a recommender system that just looks at oracle text. In order to do that I need to combine both oracle_text and oracle_text_back into one single column

In [4]:
df['oracle_combined'] = df['oracle_text_token'] + " " + df['oracle_text_back_token']

In [5]:
# start by isolating the name of the card and it's oracle_text
oracle = df['oracle_combined']

# vectorize all our words
cvec = CountVectorizer(stop_words=['none'],
                      min_df=2,
                      max_df=.98,
                      ngram_range=(2,5),
                      token_pattern="[a-zA-Z{}+'0-9-/−]+") # we should use the same RegEx to keep certain characters together 

oracle_vec = cvec.fit_transform(oracle)

# convert to a dataframe so we can use this later on as well
converted_df = pd.DataFrame(oracle_vec.toarray(), columns=cvec.get_feature_names(), index=df['name'])

In [6]:
converted_df.head()

,'{t} this,+0/+1 and,+0/+1 and has,+0/+1 counter,+0/+1 counter on,+0/+1 counters,+0/+1 counters on,+0/+1 for,+0/+1 for each,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+1 whenever,+0/+1 whenever a,+0/+2 and,+0/+2 and assigns,...,−8 target opponent gets,−8 target opponent gets an,−8 you,−8 you get,−8 you get an,−8 you get an emblem,−9 gain,−9 gain control,−9 gain control of,−9 put,−9 you,−9 you get,−9 you get an,−9 you get an emblem,−x chandra,−x return,−x return target
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Sensory Deprivation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Road of Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Storm Crow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Walking Sponge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


For the recommender system to work efficiently we need to convert the data back into a sparse matrix with the new names as the index

In [7]:
sparse_df = sparse.csr_matrix(converted_df)

In [8]:
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

In [9]:
rec.shape

(18108, 18108)

In [10]:
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,0.947247,1.0,...,1.000000,1.000000,1.000000,0.963114,1.000000,1.000000,1.0,1.000000,1.0,0.984595,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.0
Sensory Deprivation,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,0.819813,1.0,1.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000000,0.976998,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.0
Road of Return,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.991151,1.0,1.0,1.000000,1.0,0.935876,1.000000,0.982848,0.963614,1.0,...,0.957119,1.000000,0.992186,1.000000,1.000000,0.985146,1.0,0.951921,1.0,1.000000,1.0,0.856439,1.0,1.000000,1.000000,1.000000,1.0
Storm Crow,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.986143,1.0,1.0,1.000000,1.0,1.000000,0.977249,1.000000,0.943020,1.0,...,1.000000,0.979562,1.000000,0.867197,0.984492,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.0
Walking Sponge,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,1.0,1.0,1.000000,1.0,0.983648,0.933317,1.000000,1.000000,1.0,...,1.000000,0.880192,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,0.914405,0.980698,0.913836,1.0


In [11]:
# now to test the recommender system
rec_df['Shock'].sort_values()[1:11]

name
Tarfire              0.118083
Unyaro Bee Sting     0.147987
Moonglove Extract    0.222222
Goblin Test Pilot    0.242967
Ember Hauler         0.242967
Deadapult            0.242967
Crackling Triton     0.262135
Tar Pitcher          0.262135
Orcish Vandal        0.262135
Magma Jet            0.262135
Name: Shock, dtype: float64

In [12]:
rec_df['Static Orb'].sort_values()[1:11]

name
Winter Orb        0.339828
Stoic Angel       0.533431
Damping Field     0.608003
Imi Statue        0.608003
Dovin Baan        0.636490
Mungha Wurm       0.828571
Storage Matrix    0.845697
Castle Raptors    0.847279
Stabilizer        0.857143
Giant Tortoise    0.870781
Name: Static Orb, dtype: float64

In [13]:
rec_df['Prized Amalgam'].sort_values()[1:11]

name
Bone Dragon               0.463599
Footsteps of the Goryo    0.527900
Reassembling Skeleton     0.528050
Despoiler of Souls        0.538991
Scrapheap Scrounger       0.544383
Drownyard Temple          0.547321
Ghoulsteed                0.547321
Advanced Stitchwing       0.564410
Apprentice Necromancer    0.589638
Wake the Dead             0.598464
Name: Prized Amalgam, dtype: float64

In [14]:
rec_df['Wrath of God'].sort_values()[1:11]

name
Damnation          0.000000
Perish             0.351819
Winds of Rath      0.469670
Kirtar's Wrath     0.498035
Planar Collapse    0.500000
Plague Wind        0.537090
March of Souls     0.539821
Decree of Pain     0.585977
Day of Judgment    0.591752
Rout               0.611078
Name: Wrath of God, dtype: float64

This is a great start! Now I want to add the numerical features and see the results
___

## Adding numerical data to our features

In [15]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
card_faces                 object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back

In [16]:
# we already have a df for the oracle text so we can just use that one and add to it.
converted_df['cmc'] = df['cmc'].values
converted_df['activated_ability'] = df['activated_ability'].values
converted_df['triggered_ability'] = df['triggered_ability'].values

# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.898290,0.923837,0.883840,0.887452,0.628609,0.785099,0.964907,0.672239,0.873845,0.853421,0.874734,0.878754,0.971038,0.819784,0.818056,0.856161,...,0.900829,0.899128,0.881229,0.869147,0.854329,0.798255,0.916016,0.889801,0.798979,0.903114,0.844774,0.783809,0.606081,0.937124,0.761249,0.878070,0.789441
Sensory Deprivation,0.898290,0.000000,0.950078,0.923861,0.926229,0.756568,0.859141,0.976998,0.785166,0.917310,0.743795,0.917893,0.920528,0.981017,0.881875,0.910557,0.905719,...,0.934997,0.933882,0.922150,0.914231,0.904518,0.867764,0.944952,0.927768,0.868238,0.945567,0.898255,0.858295,0.741801,0.958787,0.843508,0.920080,0.861987
Road of Return,0.923837,0.950078,0.000000,0.942985,0.944758,0.817710,0.894520,0.974163,0.839125,0.938079,0.928055,0.938516,0.880978,0.985785,0.896802,0.899534,0.929400,...,0.910761,0.950489,0.934417,0.951830,0.928500,0.888600,0.958778,0.900836,0.901333,0.959239,0.923810,0.777162,0.806653,0.969138,0.882814,0.940153,0.896652
Storm Crow,0.883840,0.923861,0.942985,0.000000,0.915747,0.721981,0.839128,0.960594,0.754642,0.905562,0.890272,0.906227,0.909237,0.956639,0.865092,0.846774,0.892324,...,0.925762,0.905610,0.911089,0.804090,0.877321,0.848976,0.937131,0.917506,0.849518,0.937833,0.883799,0.838161,0.705116,0.952931,0.821273,0.908725,0.842378
Walking Sponge,0.887452,0.926229,0.944758,0.915747,0.000000,0.730626,0.844130,0.961820,0.722650,0.908498,0.893685,0.894000,0.882745,0.915975,0.869287,0.901026,0.895672,...,0.928070,0.817090,0.913854,0.928818,0.881136,0.853672,0.939086,0.920071,0.854197,0.939766,0.887413,0.843194,0.714286,0.863185,0.812400,0.823126,0.847279


In [17]:
# now to test the recommender system
rec_df['Shock'].sort_values()[1:11]

name
Tarfire                0.111477
Magma Jet              0.250731
Ember Hauler           0.250731
Unyaro Bee Sting       0.258323
Moonglove Extract      0.262957
Orcish Vandal          0.265870
Deadapult              0.275776
Explosive Apparatus    0.282453
Goblin Test Pilot      0.287948
Crackling Triton       0.287948
Name: Shock, dtype: float64

In [18]:
rec_df['Static Orb'].sort_values()[1:11]

name
Winter Orb            0.305952
Stoic Angel           0.418682
Damping Field         0.499399
Imi Statue            0.499399
Dovin Baan            0.558457
Stabilizer            0.588946
Castle Raptors        0.600509
Armored Wolf-Rider    0.606081
Royal Falcon          0.606081
Alpha Myr             0.606081
Name: Static Orb, dtype: float64

In [19]:
rec_df['Prized Amalgam'].sort_values()[1:11]

name
Bone Dragon               0.438616
Footsteps of the Goryo    0.477674
Reassembling Skeleton     0.490087
Despoiler of Souls        0.504859
Scrapheap Scrounger       0.510044
Ghoulsteed                0.512485
Advanced Stitchwing       0.521772
Stitchwing Skaab          0.539938
Splendid Reclamation      0.544561
Chronosavant              0.549436
Name: Prized Amalgam, dtype: float64

In [20]:
rec_df['Wrath of God'].sort_values()[1:11]

name
Damnation              1.110223e-16
Perish                 2.111065e-01
Winds of Rath          2.232762e-01
Plague Wind            2.358599e-01
Day of Judgment        2.524550e-01
Extinguish All Hope    2.951452e-01
Soulscour              3.071714e-01
Plague Spores          3.124761e-01
Plasma Elemental       3.134562e-01
Final Judgment         3.134562e-01
Name: Wrath of God, dtype: float64

In [21]:
rec_df['Jace, the Mind Sculptor'].sort_values()[1:11]

name
Brainstorm                0.450524
Riverwise Augur           0.454100
Coral Fighters            0.461954
Dream Cache               0.524370
Eye Spy                   0.525424
Voyage's End              0.528485
Cavalier of Gales         0.530216
Precognition              0.555188
Stormcaller of Keranos    0.568001
Watchful Automaton        0.568001
Name: Jace, the Mind Sculptor, dtype: float64

___

## Non-numerical data
Now I have to convert the non-numerical data into numerical data

In [22]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
card_faces                 object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back

### colors

In [68]:
df.head()

,layout,colors,color_identity,mana_cost,cmc,type_line,card_type,super_type,sub_type,oracle_text,oracle_text_token,legalities,rarity,power,toughness,loyalty,card_faces,activated_ability,triggered_ability,oracle_text_back,oracle_text_back_token,colors_back,power_back,toughness_back,loyalty_back,card_type_back,super_type_back,sub_type_back,mana_cost_back,oracle_combined
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,normal,[],[],{3},3.0,Artifact,Artifact,NONE,NONE,"As long as Static Orb is untapped, players can...",as long as static orb is untapped players can'...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,as long as static orb is untapped players can'...
Sensory Deprivation,normal,['U'],['U'],{U},1.0,Enchantment — Aura,Enchantment,NONE,Aura,Enchant creature Enchanted creature gets -3/-0.,enchant creature enchanted creature gets -3/-0,pioneer modern legacy pauper vintage penny com...,common,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,enchant creature enchanted creature gets -3/-0...
Road of Return,normal,['G'],['G'],{G}{G},2.0,Sorcery,Sorcery,NONE,NONE,Choose one — • Return target permanent card fr...,choose one return target permanent card from y...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,choose one return target permanent card from y...
Storm Crow,normal,['U'],['U'],{1}{U},2.0,Creature — Bird,Creature,NONE,Bird,Flying (This creature can't be blocked except ...,flying this creature can't be blocked except b...,modern legacy pauper vintage penny commander duel,common,1,2,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,flying this creature can't be blocked except b...
Walking Sponge,normal,['U'],['U'],{1}{U},2.0,Creature — Sponge,Creature,NONE,Sponge,{T}: Target creature loses your choice of flyi...,{t} target creature loses your choice of flyin...,legacy vintage commander duel,uncommon,1,1,NONE,NONE,1.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,{t} target creature loses your choice of flyin...


In [23]:
df = df.set_index(df['name']).drop(columns='name')

In [69]:
# essentially makeing dummy variables for non-numerical data

# initialize some lists to check
wburg = ['B', 'G', 'R', 'U', 'W']

card_types = ['Creature', 'Instant', 'Enchantment', 'Sorcery', 'Artifact', 'Land', 'Planeswalker', 'Tribal']

subtypes = list(set(" ".join(df['sub_type'].value_counts().index).split()))
subtypes.remove('NONE')

super_types = list(set(" ".join(df['super_type'].value_counts().index).split()))
super_types.remove('NONE')

formats = list(set(" ".join(df['legalities'].value_counts().index).split()))
formats.remove('NONE')

rarities = ['common', 'uncommon', 'rare', 'mythic']


# timer and progress checker
t0 = time.time()
counter = 0

for index in converted_df.index:
    # colors
    for color in range(len(wburg)):
        converted_df.loc[index, 'colors_' + wburg[color]] = (wburg[color] in df.loc[index, 'colors']) * 1
        converted_df.loc[index, 'color_identity_' + wburg[color]] = (wburg[color] in df.loc[index, 'color_identity']) * 1
    
    # card_types
    for ctype in range(len(card_types)):
        converted_df.loc[index, 'card_type_' + card_types[ctype]] = (card_types[ctype] in df.loc[index, 'card_type']) * 1
        
    # sub_types
    for stype in range(len(subtypes)):
        converted_df.loc[index, 'sub_type_' + subtypes[stype]] = (subtypes[stype] in df.loc[index, 'sub_type']) * 1
        
    # super_type
    for st in range(len(super_types)):
        converted_df.loc[index, 'super_type_' + super_types[st]] = (super_types[st] in df.loc[index, 'super_type']) * 1
    
    # legalities
    for form in range(len(formats)):
        converted_df.loc[index, 'legalities_' + formats[form]] = (formats[form] in df.loc[index, 'legalities']) * 1
    
    # rarity
    for r in range(len(rarities)):
        converted_df.loc[index, 'rarity_' + rarities[r]] = (rarities[r] in df.loc[index, 'rarity']) * 1
    
    # progress checker
    counter += 1
    if counter % 1000 == 0:
        print(f'Converted {counter} cards out of a total of {len(converted_df.index)}')
        print(f'mins: {(time.time() - t0)/60}')
        print('-------------------------------')

Converted 1000 cards out of a total of 18108
mins: 3.6666555364926654
Converted 2000 cards out of a total of 18108
mins: 7.262253618240356
Converted 3000 cards out of a total of 18108
mins: 10.892178817590077
Converted 4000 cards out of a total of 18108
mins: 14.577157501379649
Converted 5000 cards out of a total of 18108
mins: 18.363931651910146
Converted 6000 cards out of a total of 18108
mins: 22.24658043384552
Converted 7000 cards out of a total of 18108
mins: 26.178600804011026
Converted 8000 cards out of a total of 18108
mins: 30.450654900074007
Converted 9000 cards out of a total of 18108
mins: 34.229049619038896
Converted 10000 cards out of a total of 18108
mins: 38.032854568958285
Converted 11000 cards out of a total of 18108
mins: 41.75997579892476
Converted 12000 cards out of a total of 18108
mins: 45.566112267971036
Converted 13000 cards out of a total of 18108
mins: 49.34195979833603
Converted 14000 cards out of a total of 18108
mins: 53.355168116092685
Converted 15000 car

### power

In [31]:
df['power'].value_counts()

NONE    8357
2       3038
1       2140
3       1796
4       1019
5        603
0        457
6        331
7        123
*        119
8         56
10        23
9         23
12         7
11         5
13         3
1+*        3
15         2
-1         2
16         1
Name: power, dtype: int64

### toughness

In [32]:
df['toughness'].value_counts()

NONE    8357
2       2557
1       2333
3       1964
4       1311
5        691
6        366
0        151
7        138
*        103
8         62
9         24
10        22
11         7
12         7
13         5
1+*        5
15         2
16         1
14         1
-1         1
Name: toughness, dtype: int64

### loyalty

In [33]:
df['loyalty'].value_counts()

NONE    17919
5          64
4          62
3          42
6           9
7           7
2           4
X           1
Name: loyalty, dtype: int64

In [79]:
# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.834641,0.871781,0.835867,0.837264,0.630085,0.753724,0.926829,0.675000,0.811129,0.817173,0.839884,0.829217,0.937973,0.784763,0.788000,0.802358,...,0.869054,0.851652,0.857143,0.830407,0.814338,0.744180,0.875620,0.856369,0.772516,0.848477,0.798077,0.757603,0.747409,0.903846,0.730732,0.851341,0.750000
Sensory Deprivation,0.834641,0.000000,0.894264,0.702225,0.778569,0.638127,0.784438,0.841965,0.659832,0.740413,0.553495,0.813790,0.870901,0.906224,0.699624,0.739583,0.701193,...,0.817257,0.820575,0.838015,0.819011,0.818374,0.795243,0.811956,0.816258,0.746583,0.885459,0.840092,0.749257,0.618119,0.840092,0.787602,0.859531,0.659832
Road of Return,0.871781,0.894264,0.000000,0.902045,0.902879,0.772740,0.854956,0.945414,0.776199,0.859102,0.895085,0.908118,0.827094,0.947117,0.864135,0.871501,0.882046,...,0.887285,0.889332,0.900089,0.925578,0.877790,0.842835,0.907212,0.859882,0.874955,0.905803,0.892403,0.743032,0.838485,0.942615,0.851527,0.902962,0.850799
Storm Crow,0.835867,0.702225,0.902045,0.000000,0.794864,0.679995,0.782147,0.868235,0.658603,0.765528,0.763630,0.831806,0.859044,0.906919,0.739121,0.735549,0.750864,...,0.830704,0.828585,0.849936,0.748495,0.816443,0.810311,0.843214,0.825911,0.786193,0.893888,0.838392,0.758775,0.658856,0.878794,0.778636,0.873121,0.684865
Walking Sponge,0.837264,0.778569,0.902879,0.794864,0.000000,0.728044,0.797501,0.869357,0.635471,0.803289,0.809584,0.820409,0.834833,0.870796,0.793073,0.793000,0.752984,...,0.853127,0.752791,0.860512,0.864927,0.840755,0.811925,0.857503,0.861914,0.800483,0.894791,0.859797,0.774115,0.699342,0.829753,0.780520,0.806463,0.739622


In [80]:
# now to test the recommender system
rec_df['Shock'].sort_values()[1:11]

name
Tarfire                0.186750
Ember Hauler           0.208054
Magma Jet              0.211189
Explosive Apparatus    0.275286
Orcish Vandal          0.296268
Seal of Fire           0.305287
Moonglove Extract      0.305287
Reality Hemorrhage     0.311976
Crackling Triton       0.319664
Goblin Test Pilot      0.334297
Name: Shock, dtype: float64

In [86]:
rec_df['Lightning Bolt'].sort_values()[1:11]

name
Searing Spear            0.147426
Volcanic Hammer          0.152421
Precision Bolt           0.154846
Ghostfire                0.173666
Lightning Strike         0.178416
Open Fire                0.210648
Valakut Invoker          0.229946
Mudbutton Torchrunner    0.292370
Lightning Helix          0.354025
Pitchburn Devils         0.365925
Name: Lightning Bolt, dtype: float64

In [81]:
rec_df['Static Orb'].sort_values()[1:11]

name
Winter Orb           0.275235
Stoic Angel          0.443785
Imi Statue           0.457391
Damping Field        0.502068
Stabilizer           0.530903
Eon Hub              0.543046
Dovin Baan           0.558995
Djinn of the Lamp    0.573254
Jhovall Queen        0.576104
Trained Orgg         0.576886
Name: Static Orb, dtype: float64

In [82]:
rec_df['Prized Amalgam'].sort_values()[1:11]

name
Bone Dragon               0.426414
Despoiler of Souls        0.470232
Footsteps of the Goryo    0.471832
Ghoulsteed                0.484974
Scrapheap Scrounger       0.485386
Advanced Stitchwing       0.492719
Reassembling Skeleton     0.493427
Stitchwing Skaab          0.508251
Splendid Reclamation      0.532153
Apprentice Necromancer    0.534358
Name: Prized Amalgam, dtype: float64

In [83]:
rec_df['Wrath of God'].sort_values()[1:11]

name
Damnation              0.046512
Day of Judgment        0.193053
Winds of Rath          0.202037
Plague Wind            0.250692
Final Judgment         0.259293
Perish                 0.267708
Mass Calcify           0.275632
Soulscour              0.282707
End Hostilities        0.295639
Extinguish All Hope    0.295639
Name: Wrath of God, dtype: float64

In [84]:
rec_df['Jace, the Mind Sculptor'].sort_values()[1:11]

name
Brainstorm                0.456306
Riverwise Augur           0.462327
Coral Fighters            0.467176
Eye Spy                   0.521603
Voyage's End              0.521947
Dream Cache               0.522192
Cavalier of Gales         0.524171
Precognition              0.547323
Watchful Automaton        0.567390
Stormcaller of Keranos    0.571414
Name: Jace, the Mind Sculptor, dtype: float64

In [87]:
rec_df['Delver of Secrets // Insectile Aberration'].sort_values()[1:11]

name
Think Tank                                        0.338887
Geist of the Archives                             0.458835
Into the Wilds                                    0.484198
Rummaging Wizard                                  0.502904
Aberrant Researcher // Perfected Form             0.508602
Puresight Merrow                                  0.508772
Etherwrought Page                                 0.509602
Precognition Field                                0.520107
Galvanoth                                         0.521254
Search for Azcanta // Azcanta, the Sunken Ruin    0.527381
Name: Delver of Secrets // Insectile Aberration, dtype: float64